# Detecton of sugarbeet plants with YOLO11n

### getting stated
this notebook guides you in several stef to create and apply a detector for Sugarbeets in images. 
You will need the dataset which was created from Fendt GmbH.

important steps in this Notebook
- prepare the given dataset for YOLO
- split the data and move it to seperate folders
- train the model
- apply the model 

### 1. Make txt-files from json-files

Note: copy the sugarbeet dataset in workspace. Directory should be in the original form:

```
|datensatz_sugarbeet/
|   ├── annotations/
|   └── img/
```

skript creates folder convertedSugarbeetDataset and convertes json files to TXT files according to the requirements of YOLO

In [3]:
import os
import json

annotations_dir = "datensatz_sugarbeet/annotations"  # directory for JSON-files
output_dir = "datensatz_sugarbeet/convertedSugarbeetDataset"  # directory for YOLO-TXT-files

# define classes 
CLASSES = {
    "sugarbeet": 0,
    #"weed": 1,
    #"rock": 2,
}

# create output directory if doesnt exist
os.makedirs(output_dir, exist_ok=True)


def convert_to_yolo_format(img_width, img_height, bbox):
    """convert bounding box coordinates in YOLO format.
    
    inputparameters:    img_width           int                                        width of image
                        img_height          int                                        height of image
                        bbox                list: [[x_min, y_min],[x_max, y_max]]      absolute coordinates according to image width and height.
                        
    return:             x_center            int                                        relative x-coordinate of bounding box
                        y_center            int                                        relative y-coordinate of bounding box
                        width               int                                        with of bounding box
                        height              int                                        height  of bounding box"""
    
    x_min, y_min = bbox[0]
    x_max, y_max = bbox[1]

    # calculate YOLO-coordinates (relativ to image size)
    x_center = ((x_min + x_max) / 2) / img_width
    y_center = ((y_min + y_max) / 2) / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height

    return x_center, y_center, width, height


def process_json_file(json_file):
    """read JSON file and convert bounding box coordinates in YOLO format.

    inputparameters:       json_file           string          path to json file
    
    returns:               yolo_lines          list            list of strings with content of classes and boundingboxes in YOLO-Format"""

    with open(json_file, "r") as f:
        data = json.load(f)

    img_width = data["imgWidth"]
    img_height = data["imgHeight"]
    objects = data["objects"]

    yolo_lines = []
    for obj in objects:
        label = obj["label"]
        if label in CLASSES:
            class_id = CLASSES[label]
            bbox = obj["bbox"]
            yolo_bbox = convert_to_yolo_format(img_width, img_height, bbox)
            yolo_line = f"{class_id} {' '.join(map(str, yolo_bbox))}"
            yolo_lines.append(yolo_line)
    
    return yolo_lines


def main():
    '''iterate over all json files an create TXT Files with data for classes and bounding boxes according to the requirements of YOLO'''

    for json_file in os.listdir(annotations_dir):
        if json_file.endswith(".json"):
            json_path = os.path.join(annotations_dir, json_file)
            yolo_lines = process_json_file(json_path)

            # write data in TXT files
            txt_file = os.path.splitext(json_file)[0] + ".txt"
            txt_path = os.path.join(output_dir, txt_file)
            with open(txt_path, "w") as f:
                f.write("\n".join(yolo_lines))

if __name__ == "__main__":
    main()


### 

### 2. Move data into separate folders

split images and corresponding label files into training and validation sets.


In [ ]:
import os
import shutil
import random

# define directorys
image_dir = "datensatz_sugarbeet/img"
label_dir = "datensatz_sugarbeet/convertedSugarbeetDataset"
output_dir = "datensatz_sugarbeet/splited_dataset"

# build paths 
train_img_dir = os.path.join(output_dir, "images/train")
val_img_dir = os.path.join(output_dir, "images/val")
train_label_dir = os.path.join(output_dir, "labels/train")
val_label_dir = os.path.join(output_dir, "labels/val")

# create folder for splited data
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

split_ratio = 0.8  # 80% Training, 20% Validation

def split_data(image_dir, label_dir, train_img_dir, val_img_dir, train_label_dir, val_label_dir, split_ratio):
    '''
    Splits data randomly and move them in recently created directorys

    inputparameters:    severals paths (string) of directorys: image_dir, label_dir, train_img_dir, val_img_dir, train_label_dir, val_label_dir,
                        split_ratio:            int             defines ratio of All data and train data
    '''
    # list of all images and labels of associated labels
    images = [f for f in os.listdir(image_dir) if f.endswith((".png", ".jpg", ".jpeg"))]
    random.shuffle(images)

    # calculate number of train data
    split_index = int(len(images) * split_ratio)

    # split in train and validation data
    train_images = images[:split_index]
    val_images = images[split_index:]

    # move files
    for image_set, img_dir, label_dir_target in [
        (train_images, train_img_dir, train_label_dir),
        (val_images, val_img_dir, val_label_dir),
    ]:
        for image_file in image_set:
            # move pictures
            src_img_path = os.path.join(image_dir, image_file)
            dst_img_path = os.path.join(img_dir, image_file)
            shutil.copy(src_img_path, dst_img_path)

            # move labels
            label_file = os.path.splitext(image_file)[0] + ".txt"
            src_label_path = os.path.join(label_dir, label_file)
            if os.path.exists(src_label_path):
                dst_label_path = os.path.join(label_dir_target, label_file)
                shutil.copy(src_label_path, dst_label_path)

    print("splitting completed!")
    print(f"train data: {len(train_images)}")
    print(f"validation data: {len(val_images)}")

if __name__ == "__main__":
    split_data(image_dir, label_dir, train_img_dir, val_img_dir, train_label_dir, val_label_dir, split_ratio)


### 3. Start training

- Note: change the paths in sugarbeet_dataset.yaml to your absolute pahts to the train and validation directorys
- 100 Epochs ~ 3 hours
- skript saves the best model in runs\detect\train2\weights

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="sugarbeet_dataset.yaml", epochs=1, imgsz=640)

### 4. Detect sugarbeets with best model

- replace source with image or directory of images
- model saves copy of pictures with bounding boxes in runs\detect\predict
- prints predected bounding boxes
- plot image with sugarbeet predictions

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load the trained model
model = YOLO("runs/detect/train/weights/best.pt")
img_path = "C:/Users/Philip/Documents/Fendt_KI_Challenge/Test/images"

results = model.predict(source="C:/Users/Philip/Documents/Fendt_KI_Challenge/Test/images/sugarbeet_bbch12_000138.png", save=False, imgsz=640)

# Access predictions
for result in results:
    boxes = result.boxes.xyxy.cpu().numpy()  # all Bounding-Box-coordinates
    classes = result.boxes.cls.cpu().numpy()  # class-IDs
    sugarbeet_boxes = boxes[classes == 0] # all bounding boxes of sugarbeets
    print("Sugarbeet Boxes:", sugarbeet_boxes)

# plot image with predicted boundingboxes
image = mpimg.imread('runs/detect/predict6/sugarbeet_bbch12_000138.jpg')
plt.imshow(image)
plt.axis('off')
plt.show()
